In [1]:
path = '../../../../data/EIA/electricity_demand_ciso.parquet'

In [2]:
import pandas as pd
df = pd.read_parquet(path)
df

,period,subba,subba-name,parent,parent-name,value,value-units
0,2025-05-11 00:00:00-07:00,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,12053,megawatthours
1,2025-05-11 00:00:00-07:00,SCE,Southern California Edison,CISO,California Independent System Operator,13502,megawatthours
...,...,...,...,...,...,...,...
222451,2018-12-31 17:00:00-07:00,SDGE,San Diego Gas and Electric - CISO,CISO,California Independent System Operator,2414,megawatthours
222452,2018-12-31 17:00:00-07:00,VEA,Valley Electric Association - CISO,CISO,California Independent System Operator,75,megawatthours


In [3]:
df['period'] = pd.to_datetime(df['period'])
df['period'] = df['period'].dt.tz_convert('America/Los_Angeles')
df = df[['period', 'subba', 'value']].copy()
df.columns = ['period', 'subba', 'energy']
df.energy = df.energy.astype(float)
df.set_index('period', inplace=True)

df

,subba,energy
period,,
2025-05-11 00:00:00-07:00,PGAE,12053.0
2025-05-11 00:00:00-07:00,SCE,13502.0
...,...,...
2018-12-31 16:00:00-08:00,SDGE,2414.0
2018-12-31 16:00:00-08:00,VEA,75.0


In [8]:
dfp = df.pivot(columns='subba', values='energy').sort_index()

In [9]:
dfp

subba,PGAE,SCE,SDGE,VEA
period,,,,
2018-12-31 16:00:00-08:00,10440.0,10287.0,2414.0,75.0
2018-12-31 17:00:00-08:00,11256.0,11039.0,2566.0,86.0
...,...,...,...,...
2025-05-10 23:00:00-07:00,12846.0,14642.0,2857.0,72.0
2025-05-11 00:00:00-07:00,12053.0,13502.0,2613.0,65.0


In [10]:
dfp.to_parquet(path.replace('.parquet', '_preprocessed.parquet'))

In [5]:
path = '../../../../data/EIA/energy_generation_2024.parquet'

In [9]:
import pandas as pd
df = pd.read_parquet(path).query('respondent == "US48"')
df

,period,respondent,respondent-name,fueltype,type-name,value,value-units
8,2024-12-31T00,US48,United States Lower 48,BAT,Battery storage,3140,megawatthours
9,2024-12-31T00,US48,United States Lower 48,COL,Coal,74661,megawatthours
...,...,...,...,...,...,...,...
145612,2024-01-01T00,US48,United States Lower 48,WAT,Hydro,31141,megawatthours
145613,2024-01-01T00,US48,United States Lower 48,WND,Wind,36489,megawatthours


In [10]:
df['period'] = pd.to_datetime(df['period'], format='%Y-%m-%dT%H')
df['period'] = df['period'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')
df = df[['period', 'type-name', 'value']].copy()
df.columns = ['period', 'technology', 'energy']
df.energy = df.energy.astype(float)
df.set_index('period', inplace=True)

df

,technology,energy
period,,
2024-12-30 19:00:00-05:00,Battery storage,3140.0
2024-12-30 19:00:00-05:00,Coal,74661.0
...,...,...
2023-12-31 19:00:00-05:00,Hydro,31141.0
2023-12-31 19:00:00-05:00,Wind,36489.0


In [11]:
df.to_parquet(path.replace('.parquet', '_preprocessed_us48.parquet'))